In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from random import *

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
crashes = pd.read_pickle('crashes_weatherstn.pkl')

In [3]:
for yr in range(2012, 2013):
    print('Creating negative samples for {}'.format(yr))
    c = crashes[crashes.year == yr]
    negative_df = pd.DataFrame()
    N = len(c)

    for i, (row, dat) in tqdm(enumerate(c.iterrows())):
        #print(i)
        year, hour, day, road, beg, end = dat['year'], dat['hour'], dat['dayofyear'], dat['LABEL'], dat['BEG_MP'], dat['END_MP'], 
        # print('hour: {} day: {} segment: {}, {}, {}'.format(hour, day, road, beg, end))


        while True:
            rnd_hour = randint(0, 24)
            testdf = c[(c.hour == rnd_hour) & (c.dayofyear == day) & \
                    ((c.LABEL == road) & (c.BEG_MP == beg) & (c.END_MP == end))]

            if(len(testdf) == 0): # found negative sample with diff hour
                #print('FOUND RANDOM HOUR')
                sample = dat.copy()
                sample.hour = rnd_hour
                negative_df = negative_df.append(sample)
                break


        while True:
            rnd_day = randint(1, 366)
            testdf = c[(c.hour == hour) & (c.dayofyear == rnd_day) & \
                    ((c.LABEL == road) & (c.BEG_MP == beg) & (c.END_MP == end))]

            if(len(testdf) == 0): # found negative sample with diff day of year
                #print('FOUND RANDOM DAYOFYEAR')
                sample = dat.copy()
                sample.rnd_day = rnd_day
                negative_df = negative_df.append(sample)
                break

        while True:
            sample = c.sample(1).copy().iloc[0]

            testdf = c[(c.hour == hour) & (c.dayofyear == day) & \
                    ((c.LABEL == sample.LABEL) & (c.BEG_MP == sample.BEG_MP) & (c.END_MP == sample.END_MP))]
            if(len(testdf) == 0): # found negative sample with diff road segment
                #print('FOUND RANDOM SEGMENT')
                negative_df = negative_df.append(sample)
                break
                
    negative_df.to_pickle('negative_{}.pkl'.format(yr))

Creating negative samples for 2012


38831it [4:33:15,  2.37it/s]
